<a href="https://colab.research.google.com/github/greyhound101/multilingial/blob/master/spanish_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [2]:
pip install transformers

In [3]:
import gc
import os
import time
import math
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import date
from transformers import *
from sklearn.metrics import *
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F


from torch import Tensor
from torch.optim import *
from torch.nn.modules.loss import *
from torch.optim.lr_scheduler import * 
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import RandomSampler

In [4]:
import pandas as pd
def load_data():
    trn=pd.read_csv('/content/gdrive/My Drive/multilingual/jigsaw-toxic-comment-train.csv.zip',usecols=['toxic','comment_text'])
    tst=pd.read_csv('/content/gdrive/My Drive/multilingual/test.csv.zip',usecols=['lang','content'])  
    sub=pd.read_csv('/content/gdrive/My Drive/multilingual/submission.csv')
    val=pd.read_csv( '/content/gdrive/My Drive/multilingual/validation.csv.zip')  
    return trn,tst,sub,val


def get_lang(val,tst,lang):
  df=pd.concat([val,tst],0)
  return df.loc[df['lang']==lang].reset_index(drop=True).drop(['id','lang'],1)


def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])


train,test,sub,val=load_data()
test.columns=['comment_text','lang']
test['toxic']=sub['toxic']
df=get_lang(val,test,'es')


In [5]:
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
class Transformer(nn.Module):
    def __init__(self, transformer, num_classes=1):
        """
        Constructor
        
        Arguments:
            model {string} -- Transformer to build the model on. Expects "camembert-base".
            num_classes {int} -- Number of classes (default: {1})
        """
        super().__init__()
        
        self.transformer = transformer

        self.nb_features = self.transformer.pooler.dense.out_features

        self.pooler = nn.Sequential(
            nn.Linear(self.nb_features, self.nb_features), 
            nn.Sigmoid(),
        )

        self.logit = nn.Linear(self.nb_features, num_classes)

    def forward(self, tokens):
        """
        Usual torch forward function
        
        Arguments:
            tokens {torch tensor} -- Sentence tokens
        
        Returns:
            torch tensor -- Class logits
        """
        hidden_states = self.transformer(
            tokens, attention_mask=(tokens > 0).long()
        )[1]

        # hidden_states = hidden_states[-1][:, 0] # Use the representation of the first token of the last layer

        ft = self.pooler(hidden_states)

        return self.logit(ft)

In [6]:
import numpy as np
model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model=Transformer(model)
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
x_train = regular_encode(list(df.comment_text.values), tokenizer, maxlen=128)
y_train = df.toxic.values

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [7]:
class bce(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(bce, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        one=(targets-1)*torch.log(1-inputs)
        zero=(targets*torch.log(inputs))
        loss = torch.mean(one+zero)
        
        return loss

In [8]:
class JigsawDataset(Dataset):
    """
    Torch dataset for training and validating
    """
    def __init__(self, x,y):
        super().__init__()
        self.y = y 
        self.sentences = x
        

    def __len__(self):
        return self.sentences.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.sentences[idx]), torch.tensor(self.y[idx])

def fit(model, train_dataset, epochs=1, batch_size=8, lr=5e-4):
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    optimizer = AdamW(model.parameters(), lr=lr)
    

    
    for epoch in range(epochs):
        model.train()
        avg_loss =0
        optimizer.zero_grad()
        lss=bce()
        for step, (x, y_batch) in tqdm(enumerate(train_loader), total=len(train_loader)): 
            
            y_pred = model(x)
            
            loss = lss(y_pred.view(-1).float(), y_batch.float())
            loss.backward()
            avg_loss += loss.item() / len(train_loader)

            optimizer.step()
            #optimizer.step()
            
            model.zero_grad()
            optimizer.zero_grad()
        print(avg_loss)

In [ ]:
train_dataset = JigsawDataset(x_train,y_train)
fit(model, train_dataset, epochs=10, batch_size=32, lr=1e-4)